In [ ]:
var dumpFilePath = await Kernel.GetInputAsync(
    "Please provide a memory dump file.",
    typeHint: "file");

In [ ]:
#r "nuget: Microsoft.Diagnostics.Runtime, 3.0.0-beta.23214.4"

using Microsoft.Diagnostics.Runtime;
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Formatting;
using System;

Formatter.Register<UIntPtr>(n => n.ToString("x16"), "text/html");
Formatter.ListExpansionLimit = int.MaxValue;

In [ ]:
public static IEnumerable<ClrObject> GetEnumerator(this ClrArray clrArray)
{
    for (var i = 0; i < clrArray.Length; i++)
    {
        yield return clrArray.GetObjectValue(i);
    }
}

public static IEnumerable<ClrObject> GetListOfTEnumerator(this ClrObject listOfTClrObject)
{
    var count = listOfTClrObject.ReadField<int>("_size");
    foreach (var item in listOfTClrObject.ReadObjectField("_items").AsArray().GetEnumerator())
    {
        if (count-- == 0)
        {
            yield break;
        }
        else
        {
            yield return item;
        }
    }
}


public static object GetSqlParameters(this ClrObject sqlParameterCollectionClrObject)
{
    if (sqlParameterCollectionClrObject.IsNull)
    {
        return new
        {
            Address = "0000000000000000",
        };
    }

    return new
    {
        Address = (UIntPtr)(sqlParameterCollectionClrObject.Address),
        IsDirty = sqlParameterCollectionClrObject.ReadField<bool>("_isDirty"),
        Parameters = from parameter in sqlParameterCollectionClrObject.ReadObjectField("_items").GetListOfTEnumerator()
                     select new
                     {
                         Address = (UIntPtr)(parameter.Address),
                         ParameterName = parameter.ReadStringField("_parameterName"),
                         Value = GetValue(parameter),
                     },
    };
    
    object GetValue(ClrObject sqlParameter)
    {
        var value = sqlParameter.ReadObjectField("_value");
        
        return value.Type.Name switch
        {
            "System.Int32" => value.ReadBoxedValue<int>(),
            "System.String" => value.AsString(),
            "System.DBNull" => "NULL",
            _ => value.ToString(),
        };
    }
}

public static string ToStorageTypeString(this int storageType)
    => storageType switch
    {
        0 => $"Empty ({storageType})",
        1 => $"Boolean ({storageType})",
        2 => $"Byte ({storageType})",
        3 => $"DateTime ({storageType})",
        4 => $"Decimal ({storageType})",
        5 => $"Double ({storageType})",
        6 => $"Int16 ({storageType})",
        7 => $"Int32 ({storageType})",
        8 => $"Int64 ({storageType})",
        9 => $"Money ({storageType})",
        10 => $"Single ({storageType})",
        11 => $"String ({storageType})",
        12 => $"SqlBinary ({storageType})",
        13 => $"SqlCachedBuffer ({storageType})",
        14 => $"SqlGuid ({storageType})",
        15 => $"SqlXml ({storageType})",
        16 => $"Date ({storageType})",
        17 => $"DateTime2 ({storageType})",
        18 => $"DateTimeOffset ({storageType})",
        19 => $"Time ({storageType})",
        _ => $"Unknown ({storageType})",
    };

public static string ToSqlBufferValueString(this ClrObject sqlBufferClrObject)
    => sqlBufferClrObject.ReadField<int>("_type") switch
    {
        0 => "<empty>",
        1 => $"{sqlBufferClrObject.ReadValueTypeField("_value").ReadField<bool>("_boolean")} (SqlBoolean)",
        //2=>,
        //3=>,
        //4=>,
        //5=>,
        //6=>,
        7 => $"{sqlBufferClrObject.ReadValueTypeField("_value").ReadField<int>("_int32")} (SqlInt32)",
        //8=>,
        //9=>,
        //10=>,
        11 => $"{sqlBufferClrObject.ReadObjectField("_object").AsString()} ({(UIntPtr)(sqlBufferClrObject.ReadObjectField("_object").Address)} SqlString)",
        //12=>,
        //13=>,
        //14=>,
        //15=>,
        //16=>,
        //17=>,
        //18=>,
        //19=>,
        _ => "???",
    };

In [ ]:
using (var dataTarget = DataTarget.LoadDump(dumpFilePath))
{
    foreach (var version in dataTarget.ClrVersions)
    {
        var clrRuntime = version.CreateRuntime();

        if (!clrRuntime.Heap.CanWalkHeap)
        {
            Console.WriteLine("Cannot walk the heap!");
            continue;
        }
                
        var sqlConnections = (from clrObject in clrRuntime.Heap.EnumerateObjects()
                              where clrObject.IsValid && !clrObject.IsFree && clrObject.Type.Name == "System.Data.SqlClient.SqlConnection"
                              select new
                              {
                                  Address = (UIntPtr)(clrObject.Address),
                                  ObjectID = clrObject.ReadField<int>("ObjectID"),
                                  OriginalConnectionId = clrObject.ReadField<Guid>("_originalConnectionId"),
                                  ReconnectCount = clrObject.ReadField<int>("_reconnectCount"),
                                  ConnectRetryCount = clrObject.ReadField<int>("_connectRetryCount"),
                                  CloseCount = clrObject.ReadField<int>("_closeCount"),
                                  AsyncCommandInProgress = clrObject.ReadField<bool>("_AsyncCommandInProgress"),
                                  InnerConnection = clrObject.ReadObjectField("_innerConnection").ToString(),
                              })
                              .OrderBy(sc => sc.ObjectID);

        sqlConnections.Display();

        var sqlCommands = (from clrObject in clrRuntime.Heap.EnumerateObjects()
                           where clrObject.IsValid && !clrObject.IsFree && clrObject.Type.Name == "System.Data.SqlClient.SqlCommand"
                           select new
                           {
                               Address = (UIntPtr)(clrObject.Address),
                               ObjectID = clrObject.ReadField<int>("ObjectID"),
                               ActiveConnection = (UIntPtr)(clrObject.ReadObjectField("_activeConnection").Address),
                               CommandText = clrObject.ReadStringField("_commandText")?.Split("\\r\\n"),
                               CommandType = $"{clrObject.ReadField<System.Data.CommandType>("_commandType")} ({clrObject.ReadField<int>("_commandType")}) (System.Data.CommandType)",
                               Parameters = clrObject.ReadObjectField("_parameters").GetSqlParameters(),
                               CachedMetaData = (UIntPtr)(clrObject.ReadObjectField("_cachedMetaData").Address),
                           })
                           .OrderBy(sc => sc.ObjectID);

        sqlCommands.Display();

        var sqlDataReaders = (from clrObject in clrRuntime.Heap.EnumerateObjects()
                              where clrObject.IsValid && !clrObject.IsFree && clrObject.Type.Name == "System.Data.SqlClient.SqlDataReader"
                              select new
                              {
                                  Address = (UIntPtr)(clrObject.Address),
                                  ObjectID = clrObject.ReadField<int>("ObjectID"),
                                  IsInitialized = clrObject.ReadField<bool>("_isInitialized"),
                                  IsClosed = clrObject.ReadField<bool>("_isClosed"),
                                  MetaDataConsumed = clrObject.ReadField<bool>("_metaDataConsumed"),
                                  HasRows = clrObject.ReadField<bool>("_hasRows"),
                                  SharedState = (UIntPtr)(clrObject.ReadObjectField("_sharedState").Address),
                                  Parser = (UIntPtr)(clrObject.ReadObjectField("_parser").Address),
                                  StateObject = (UIntPtr)(clrObject.ReadObjectField("_stateObj").Address),
                                  Command = (UIntPtr)(clrObject.ReadObjectField("_command").Address),
                                  Connection = (UIntPtr)(clrObject.ReadObjectField("_connection").Address),
                                  Data = (UIntPtr)(clrObject.ReadObjectField("_data").Address),
                                  MetaData = (UIntPtr)(clrObject.ReadObjectField("_metaData").Address),
                              })
                              .OrderBy(sdr => sdr.ObjectID);

        sqlDataReaders.Display();

        var sqlBufferArrays = (from clrObject in clrRuntime.Heap.EnumerateObjects()
                               where clrObject.IsValid && !clrObject.IsFree && clrObject.IsArray && clrObject.Type.Name == "System.Data.SqlClient.SqlBuffer[]"
                               let clrArray = clrObject.AsArray()
                               select new
                               {
                                   Address = (UIntPtr)(clrObject.Address),
                                   Items = from item in clrArray.GetEnumerator()
                                           select new
                                           {
                                               Address = (UIntPtr)(clrArray.Address),
                                               Value = item.ToSqlBufferValueString(),
                                               Type = item.ReadField<int>("_type").ToStorageTypeString(),
                                               IsNull = item.ReadField<bool>("_isNull"),
                                           }
                               }).ToList();

        sqlBufferArrays.Display();
 
        var sqlMetaDataSets = (from clrObject in clrRuntime.Heap.EnumerateObjects()
                               where clrObject.IsValid && !clrObject.IsFree && clrObject.Type.Name == "System.Data.SqlClient._SqlMetaDataSet"
                               select new
                               {
                                   Address = (UIntPtr)(clrObject.Address),
                                   Id = clrObject.ReadField<ushort>("id"),
                                   VisibleColumns = clrObject.ReadField<int>("visibleColumns"),
                                   SchemaTable = (UIntPtr)(clrObject.Address),
                                   MetaDataArray = from item in clrObject.ReadObjectField("metaDataArray").AsArray().GetEnumerator()
                                                   let type = item.ReadField<int>("type")
                                                   select new
                                                   {
                                                       Address = (UIntPtr)(item.Address),
                                                       ColumnName = item.ReadStringField("column"),
                                                       Type = $"{(System.Data.SqlDbType)type} ({type}) (System.Data.SqlDbType)",
                                                   }
                               }).ToList();

        sqlMetaDataSets.Display();
    }
}